In [1]:
import numpy as np
from tensorflow.keras import Sequential

import tensorflow.compat.v1 as tf
from tensorflow.compat.v1.keras.backend import set_session

config = tf.compat.v1.ConfigProto()

config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU

config.log_device_placement = True  # to log device placement (on which device the operation ran)

sess = tf.compat.v1.Session(config=config)

set_session(sess)




from tensorflow.keras.layers import  Flatten, Conv2D, MaxPooling2D, Dense, Dropout, Softmax, Conv2DTranspose, BatchNormalization,Activation, AveragePooling2D


def vgg16(input_shape):
    model = Sequential()
    model.add(Conv2D(64, (3,3), padding ="same", activation = "relu", input_shape=input_shape))
    model.add(Conv2D(64, (3,3), padding ="same", activation = "relu", name = "copy_crop1"))
    model.add(MaxPooling2D((2,2), strides=(2, 2)))
    model.add(Conv2D(128, (3,3), padding = "same", activation = "relu"))
    model.add(Conv2D(128, (3,3), padding = "same", activation = "relu", name = "copy_crop2"))
    model.add(MaxPooling2D((2,2), strides=(2, 2)))
    model.add(Conv2D(256, (3,3), padding = "same", activation = "relu"))
    model.add(Conv2D(256, (3,3), padding = "same", activation = "relu"))
    model.add(Conv2D(256, (3,3), padding = "same", activation = "relu", name = "copy_crop3"))
    model.add(MaxPooling2D((2,2), strides=(2, 2), name = 'block3_pool'))
    model.add(Conv2D(512, (3,3), padding = "same", activation = "relu"))
    model.add(Conv2D(512, (3,3), padding = "same", activation = "relu"))
    model.add(Conv2D(512, (3,3), padding = "same", activation = "relu", name = "copy_crop4"))
    model.add(MaxPooling2D((2,2), strides=(2, 2), name = 'block4_pool'))
    model.add(Conv2D(512, (3,3), padding = "same", activation = "relu"))
    model.add(Conv2D(512, (3,3), padding = "same", activation = "relu"))
    model.add(Conv2D(512, (3,3), padding = "same", activation = "relu"))
    model.add(MaxPooling2D((2,2), strides=(2, 2), name = "last_layer"))
    return model

def global_average_pooling(input, gap_size):
    w, h, c = (input.shape[1:])
    x = AveragePooling2D((w/gap_size, h/gap_size))(input)
    x = Conv2D(c//4, (1,1), padding="same", activation='relu')(x)
    x = BatchNormalization()(x)
    x = tf.image.resize(x, (w, h))
    return x


def contract_path(input_shape):
    input= tf.keras.layers.Input(shape = input_shape)
    x =  Conv2D(64, (3,3), padding = "same", activation = "relu")(input)
    x =  Conv2D(64, (3,3), padding = "same", activation = "relu", name = "copy_crop1")(x)
    x = MaxPooling2D((2, 2))(x)
    x =  Conv2D(128, (3,3), padding = "same", activation = "relu")(x)
    x =  Conv2D(128, (3,3), padding = "same", activation = "relu", name = "copy_crop2")(x)
    x = MaxPooling2D((2, 2))(x)
    x =  Conv2D(256, (3,3), padding = "same", activation = "relu")(x)
    x =  Conv2D(256, (3,3), padding = "same", activation = "relu", name = "copy_crop3")(x)
    x = MaxPooling2D((2, 2))(x)
    x =  Conv2D(512, (3,3), padding = "same", activation = "relu")(x)
    x =  Conv2D(512, (3,3), padding = "same", activation = "relu", name = "copy_crop4")(x)
    x = MaxPooling2D((2, 2))(x)
    x =  Conv2D(1024, (3,3), padding = "same", activation = "relu")(x)
    x =  Conv2D(1024, (3,3), padding = "same", activation = "relu", name = "last_layer")(x)
    contract_path =  tf.keras.Model(inputs = input, outputs = x)
    return contract_path

def pspunet(input_shape, n_classes):
    contract_model = contract_path(input_shape=input_shape)
    layer_names  = ["copy_crop1", "copy_crop2",  "copy_crop3" ,"copy_crop4"]
    layers = [contract_model.get_layer(name).output for name in layer_names]

    extract_model = tf.keras.Model(inputs=contract_model.input, outputs=layers)
    input= tf.keras.layers.Input(shape =input_shape)
    output_layers = extract_model(inputs = input)
    last_layer = output_layers[-1]
    
    feature_map = last_layer
    pooling_1 = global_average_pooling(feature_map, 1)
    pooling_2 = global_average_pooling(feature_map, 2)
    pooling_3 = global_average_pooling(feature_map, 3)
    pooling_4 = global_average_pooling(feature_map, 6)
    x = tf.keras.layers.Concatenate(axis=-1)([pooling_1,pooling_2,pooling_3,pooling_4])
    x =  Conv2D(256, (1,1), padding = "same")(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = tf.keras.layers.Concatenate()([x, output_layers[3]])

    x =  Conv2D(256, (3,3), padding = "same")(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x =  Conv2D(256, (3,3), padding = "same")(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2DTranspose(256, 4, (2,2), padding = "same")(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = tf.keras.layers.Concatenate()([x, output_layers[2]])

    x =  Conv2D(128, (3,3), padding = "same")(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x =  Conv2D(128, (3,3), padding = "same")(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2DTranspose(128, 4, (2,2), padding = "same")(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = tf.keras.layers.Concatenate()([x, output_layers[1]])


    x =  Conv2D(64, (3,3), padding = "same")(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x =  Conv2D(64, (3,3), padding = "same")(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)


    x = Conv2DTranspose(64, 4, (2,2), padding = "same")(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = tf.keras.layers.Concatenate()([x, output_layers[0]])

    x =  Conv2D(64, (3,3), padding = "same")(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x =  Conv2D(64, (3,3), padding = "same")(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x =  Conv2D(n_classes, (1,1), activation = "relu")(x)
    
    return tf.keras.Model(inputs = input , outputs = x)

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5



In [2]:

import tensorflow.compat.v1 as tf

from tensorflow.compat.v1.keras.backend import set_session

config = tf.compat.v1.ConfigProto()

config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU

config.log_device_placement = True  # to log device placement (on which device the operation ran)

sess = tf.compat.v1.Session(config=config)

set_session(sess)


from IPython.display import clear_output

import matplotlib.pyplot as plt


def display(display_list):
    plt.figure(figsize=(7, 7))
    for i in range(3):
        plt.subplot(3, 3, i*3+1)
        plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i][0]/255))
        plt.axis('off')
        plt.subplot(3, 3, i*3+2)
        plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i][1]/255))
        plt.axis('off')
        plt.subplot(3, 3, i*3+3)
        plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i][2]/255))
        plt.axis('off')
    plt.show()

def create_mask(pred_mask):
    pred_mask = tf.argmax(pred_mask, axis=-1)
    pred_mask = pred_mask[..., tf.newaxis]
    return pred_mask[0]   

def show_predictions(image, label, model):

    if model:
        pred_mask = [model.predict(image[tf.newaxis, ...]) for image in image]
        display_list = [[image[i], label[i], create_mask(pred_mask[i])] for i in range(3)]
        display(display_list)
    else :  
        display_list = [[image[i], label[i], image[i]] for i in range(3)]
        display(display_list)

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5



In [ ]:
import cv2
import time

import tensorflow.compat.v1 as tf

from tensorflow.compat.v1.keras.backend import set_session

config = tf.compat.v1.ConfigProto()

config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU

config.log_device_placement = True  # to log device placement (on which device the operation ran)

sess = tf.compat.v1.Session(config=config)

set_session(sess)

import numpy as np
gpus = tf.config.experimental.list_physical_devices('GPU')


if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(
       gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=3000)])
    except RuntimeError as e:
        print(e)
        
cap= cv2.VideoCapture("D:/Swayatt Robots/work/work done/dataSets/test/annoted_video.mp4")


IMG_WIDTH = 400
IMG_HEIGHT = 400
n_classes = 7

model = pspunet((IMG_HEIGHT, IMG_WIDTH ,3), n_classes)
model.load_weights("pspunet_weight.h5")

while True:
    start= time.time()
    try:
        _,frame = cap.read()
        frame = cv2.resize(frame, (IMG_WIDTH, IMG_HEIGHT))
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = frame[tf.newaxis, ...]
        frame = frame/255
    except:
        #out.release()
        cv2.destroyAllWindows()
        cap.release()
        break
        
    pre = model.predict(frame)
    pre = create_mask(pre).numpy()

    frame2 = frame/2
    #frame2 = cv2.cvtColor(frame2, cv2.COLOR_RGB2BGR)
    frame2[0][(pre==1).all(axis=2)] += [0, 0, 0] #""bike_lane_normal", "sidewalk_asphalt", "sidewalk_urethane""
    frame2[0][(pre==2).all(axis=2)] += [0.5, 0.5,0] # "caution_zone_stairs", "caution_zone_manhole", "caution_zone_tree_zone", "caution_zone_grating", "caution_zone_repair_zone"]
    frame2[0][(pre==3).all(axis=2)] += [0.2, 0.7, 0.5] #"alley_crosswalk","roadway_crosswalk"
    frame2[0][(pre==4).all(axis=2)] += [0, 0.5, 0.5] #"braille_guide_blocks_normal", "braille_guide_blocks_damaged"
    frame2[0][(pre==5).all(axis=2)] += [0, 0, 0.5] #"roadway_normal","alley_normal","alley_speed_bump", "alley_damaged""
    frame2[0][(pre==6).all(axis=2)] += [0.5, 0, 0] #"sidewalk_blocks","sidewalk_cement" , "sidewalk_soil_stone", "sidewalk_damaged","sidewalk_other"
    video = np.uint8(frame2)

    cv2.imshow("ori", frame[0])
    cv2.imshow("pre", frame2[0])
#     out.write(frame2[0])
    print(1/(time.time()-start))
    cv2.waitKey(1)

# out.release()
    

# )

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5

0.06464091821548454
3.488619142597161
4.145118029462399
4.015705497977453
3.9384874262056635
4.0299815041675675
4.192207896051974
4.025599189179851
3.862268628294178
4.256068059682106
4.036077647966759
4.108312902634948
4.043076756981851
4.3387810708792065
4.08948238744501
4.143295884367551
4.246579392319452
4.264961619144365
4.266697048827456
4.131651634612733
4.075926856312127
4.126249762664377
4.0594137408067335
4.1996249247796955
4.1503394541620615
4.065264378110031
4.151062383154677
4.208976492051291
4.043029989897978
4.30545800942944
4.171776407400039
3.9694467499552832
4.181079246343814
4.048071466349588
4.183097481846135
3.984844627025055
4.2849171428191974
4.221494647021466
3.8315203291543494
4.1319406202190345
4.094056922202939
4.100801622211946
4.172282691951878
4.202604143182786
4.229553095497879
3.861969903862887
4.2300265744